In [1]:
%cd /aiffel

import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

/aiffel


In [2]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [3]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
with open('aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [5]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 1172.04it/s]


In [6]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?
######################################################################
## chosen ##
라이언에게 말했다.
######################################################################
## rejected ##
Allow me to answer your question. I know that you are curious about me.


In [7]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [11]:
trainer.fit(use_lora=0)

model.save_pretrained('aiffel/KoChatGPT/output_2_RM')







Train epoch:   0%|          | 0/1 [00:00<?, ?it/s]






Train step of epoch 0:   0%|          | 0/250 [00:00<?, ?it/s]






Train step of epoch 0:   0%|          | 1/250 [00:01<05:03,  1.22s/it]






Train step of epoch 0:   0%|          | 1/250 [00:01<05:03,  1.22s/it, loss=0.734]






Train step of epoch 0:   1%|          | 2/250 [00:02<04:13,  1.02s/it, loss=0.734]






Train step of epoch 0:   1%|          | 2/250 [00:02<04:13,  1.02s/it, loss=0.706]






Train step of epoch 0:   1%|          | 3/250 [00:02<03:56,  1.04it/s, loss=0.706]






Train step of epoch 0:   1%|          | 3/250 [00:03<03:56,  1.04it/s, loss=0.572]






Train step of epoch 0:   2%|▏         | 4/250 [00:03<03:47,  1.08it/s, loss=0.572]






Train step of epoch 0:   2%|▏         | 4/250 [00:03<03:47,  1.08it/s, loss=1.04] 






Train step of epoch 0:   2%|▏         | 5/250 [00:04<03:42,  1.10it/s, loss=1.04]






Train step of epoch 0:   2%|▏         | 5/250 [00:04<03:42,  1.10it/s, loss=0.70

Train step of epoch 0:  15%|█▍        | 37/250 [00:33<03:13,  1.10it/s, loss=0.943]






Train step of epoch 0:  15%|█▍        | 37/250 [00:33<03:13,  1.10it/s, loss=0.474]






Train step of epoch 0:  15%|█▌        | 38/250 [00:34<03:13,  1.10it/s, loss=0.474]






Train step of epoch 0:  15%|█▌        | 38/250 [00:34<03:13,  1.10it/s, loss=0.42] 






Train step of epoch 0:  16%|█▌        | 39/250 [00:35<03:12,  1.10it/s, loss=0.42]






Train step of epoch 0:  16%|█▌        | 39/250 [00:35<03:12,  1.10it/s, loss=0.634]






Train step of epoch 0:  16%|█▌        | 40/250 [00:36<03:11,  1.10it/s, loss=0.634]






Train step of epoch 0:  16%|█▌        | 40/250 [00:36<03:11,  1.10it/s, loss=0.52] 






Train step of epoch 0:  16%|█▋        | 41/250 [00:37<03:10,  1.10it/s, loss=0.52]






Train step of epoch 0:  16%|█▋        | 41/250 [00:37<03:10,  1.10it/s, loss=0.813]






Train step of epoch 0:  17%|█▋        | 42/250 [00:38<03:08,  1.10it/s, loss=0.813]






Train step o

Train step of epoch 0:  29%|██▉       | 73/250 [01:05<02:37,  1.12it/s, loss=0.571]






Train step of epoch 0:  30%|██▉       | 74/250 [01:06<02:36,  1.13it/s, loss=0.571]






Train step of epoch 0:  30%|██▉       | 74/250 [01:06<02:36,  1.13it/s, loss=0.656]






Train step of epoch 0:  30%|███       | 75/250 [01:07<02:35,  1.12it/s, loss=0.656]






Train step of epoch 0:  30%|███       | 75/250 [01:07<02:35,  1.12it/s, loss=0.711]






Train step of epoch 0:  30%|███       | 76/250 [01:08<02:34,  1.12it/s, loss=0.711]






Train step of epoch 0:  30%|███       | 76/250 [01:08<02:34,  1.12it/s, loss=0.764]






Train step of epoch 0:  31%|███       | 77/250 [01:09<02:33,  1.13it/s, loss=0.764]






Train step of epoch 0:  31%|███       | 77/250 [01:09<02:33,  1.13it/s, loss=0.636]






Train step of epoch 0:  31%|███       | 78/250 [01:10<02:32,  1.13it/s, loss=0.636]






Train step of epoch 0:  31%|███       | 78/250 [01:10<02:32,  1.13it/s, loss=0.558]






Train step

Train step of epoch 0:  44%|████▍     | 110/250 [01:38<02:04,  1.12it/s, loss=0.438]






Train step of epoch 0:  44%|████▍     | 110/250 [01:38<02:04,  1.12it/s, loss=0.589]






Train step of epoch 0:  44%|████▍     | 111/250 [01:39<02:03,  1.12it/s, loss=0.589]






Train step of epoch 0:  44%|████▍     | 111/250 [01:39<02:03,  1.12it/s, loss=0.517]






Train step of epoch 0:  45%|████▍     | 112/250 [01:40<02:02,  1.12it/s, loss=0.517]






Train step of epoch 0:  45%|████▍     | 112/250 [01:40<02:02,  1.12it/s, loss=1.17] 






Train step of epoch 0:  45%|████▌     | 113/250 [01:41<02:01,  1.12it/s, loss=1.17]






Train step of epoch 0:  45%|████▌     | 113/250 [01:41<02:01,  1.12it/s, loss=0.847]






Train step of epoch 0:  46%|████▌     | 114/250 [01:42<02:01,  1.12it/s, loss=0.847]






Train step of epoch 0:  46%|████▌     | 114/250 [01:42<02:01,  1.12it/s, loss=0.774]






Train step of epoch 0:  46%|████▌     | 115/250 [01:43<02:00,  1.12it/s, loss=0.774]








Train step of epoch 0:  58%|█████▊    | 146/250 [02:10<01:33,  1.11it/s, loss=0.831]






Train step of epoch 0:  59%|█████▉    | 147/250 [02:11<01:32,  1.12it/s, loss=0.831]






Train step of epoch 0:  59%|█████▉    | 147/250 [02:11<01:32,  1.12it/s, loss=0.831]






Train step of epoch 0:  59%|█████▉    | 148/250 [02:12<01:31,  1.11it/s, loss=0.831]






Train step of epoch 0:  59%|█████▉    | 148/250 [02:12<01:31,  1.11it/s, loss=0.582]






Train step of epoch 0:  60%|█████▉    | 149/250 [02:13<01:30,  1.12it/s, loss=0.582]






Train step of epoch 0:  60%|█████▉    | 149/250 [02:13<01:30,  1.12it/s, loss=0.679]






Train step of epoch 0:  60%|██████    | 150/250 [02:14<01:29,  1.12it/s, loss=0.679]






Train step of epoch 0:  60%|██████    | 150/250 [02:14<01:29,  1.12it/s, loss=0.628]






Train step of epoch 0:  60%|██████    | 151/250 [02:15<01:28,  1.12it/s, loss=0.628]






Train step of epoch 0:  60%|██████    | 151/250 [02:15<01:28,  1.12it/s, loss=0.701]







Train step of epoch 0:  73%|███████▎  | 183/250 [02:43<00:59,  1.12it/s, loss=0.592]






Train step of epoch 0:  73%|███████▎  | 183/250 [02:44<00:59,  1.12it/s, loss=0.409]






Train step of epoch 0:  74%|███████▎  | 184/250 [02:44<00:58,  1.12it/s, loss=0.409]






Train step of epoch 0:  74%|███████▎  | 184/250 [02:44<00:58,  1.12it/s, loss=1.2]  






Train step of epoch 0:  74%|███████▍  | 185/250 [02:45<00:57,  1.12it/s, loss=1.2]






Train step of epoch 0:  74%|███████▍  | 185/250 [02:45<00:57,  1.12it/s, loss=0.49]






Train step of epoch 0:  74%|███████▍  | 186/250 [02:46<00:57,  1.12it/s, loss=0.49]






Train step of epoch 0:  74%|███████▍  | 186/250 [02:46<00:57,  1.12it/s, loss=0.455]






Train step of epoch 0:  75%|███████▍  | 187/250 [02:47<00:56,  1.12it/s, loss=0.455]






Train step of epoch 0:  75%|███████▍  | 187/250 [02:47<00:56,  1.12it/s, loss=0.926]






Train step of epoch 0:  75%|███████▌  | 188/250 [02:48<00:55,  1.12it/s, loss=0.926]






Tra

Train step of epoch 0:  88%|████████▊ | 219/250 [03:16<00:27,  1.12it/s, loss=0.499]






Train step of epoch 0:  88%|████████▊ | 220/250 [03:17<00:26,  1.12it/s, loss=0.499]






Train step of epoch 0:  88%|████████▊ | 220/250 [03:17<00:26,  1.12it/s, loss=0.477]






Train step of epoch 0:  88%|████████▊ | 221/250 [03:17<00:25,  1.12it/s, loss=0.477]






Train step of epoch 0:  88%|████████▊ | 221/250 [03:17<00:25,  1.12it/s, loss=0.351]






Train step of epoch 0:  89%|████████▉ | 222/250 [03:18<00:25,  1.12it/s, loss=0.351]






Train step of epoch 0:  89%|████████▉ | 222/250 [03:18<00:25,  1.12it/s, loss=0.549]






Train step of epoch 0:  89%|████████▉ | 223/250 [03:19<00:24,  1.12it/s, loss=0.549]






Train step of epoch 0:  89%|████████▉ | 223/250 [03:19<00:24,  1.12it/s, loss=0.524]






Train step of epoch 0:  90%|████████▉ | 224/250 [03:20<00:23,  1.12it/s, loss=0.524]






Train step of epoch 0:  90%|████████▉ | 224/250 [03:20<00:23,  1.12it/s, loss=0.731]







In [12]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: 0.2


In [13]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: 0.5


In [14]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: 0.7


In [15]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: 0.5


In [16]:
torch.cuda.empty_cache()